# 02_train_resnet50.ipynb

**Goal:** Evaluate the trained baseline ResNet-50 model using ROC, AUC, precision–recall, and confusion matrix visualizations.

In [28]:
# Add script to include project root in sys.path

import sys
from pathlib import Path

# Add project root (one level up from notebooks/)
project_root = Path(__file__).resolve().parent.parent if "__file__" in globals() else Path.cwd().parent
sys.path.append(str(project_root))

In [29]:
# Import necessary libraries

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve, average_precision_score
from src.model import build_resnet50_baseline
from src.data_loader import get_data_loader
from pathlib import Path


In [30]:
# Load the trained model

import torch
from pathlib import Path

# --- Detect project root (so notebook paths always work) ---
# Go up until you find the project folder containing 'saved_models'
project_root = Path.cwd()
while project_root.name != "AI-Assisted-Pneumonia-Detection-Project" and project_root.parent != project_root:
    project_root = project_root.parent
print(f"Project root detected: {project_root}")

# Make sure saved_models folder exists
saved_dir = project_root / "saved_models"
if not saved_dir.exists():
    raise FileNotFoundError(f" Folder not found: {saved_dir.resolve()}")

# Find latest model file
model_files = list(saved_dir.glob("*.pt"))
if not model_files:
    raise FileNotFoundError(" No model checkpoint (.pt) files found in saved_models/")
latest_model = max(model_files, key=lambda f: f.stat().st_mtime)
print(f"Loading latest model: {latest_model.name}")

# --- Load model ---
from src.model import build_resnet50_baseline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = build_resnet50_baseline()
model.load_state_dict(torch.load(latest_model, map_location=device, weights_only=True))
model.to(device)
model.eval()
print("Model loaded and ready for evaluation.")

# --- Prepare data loader ---
from src.data_loader import get_data_loader

# Detect the correct CSV file dynamically
csv_candidates = list((project_root / "data/rsna_subset").glob("*.csv"))
if not csv_candidates:
    raise FileNotFoundError("No CSV files found in data/rsna_subset/")
else:
    csv_path = csv_candidates[0]  # e.g., stage_2_train_labels.csv
    print(f"Using label file: {csv_path.name}")

img_dir = project_root / "data/rsna_subset/train_images"
if not img_dir.exists():
    raise FileNotFoundError(f"❌ Image directory not found: {img_dir}")

loader = get_data_loader(csv_path, img_dir, batch_size=8)
print("Data loader initialized successfully.")


Project root detected: /Users/adrianadewunmi/VSCODE/AI-Assisted-Pneumonia-Detection-Project
Loading latest model: resnet50_baseline.pt
Model loaded and ready for evaluation.
Using label file: stage_2_train_labels.csv
Loaded 30227 records from /Users/adrianadewunmi/VSCODE/AI-Assisted-Pneumonia-Detection-Project/data/rsna_subset/stage_2_train_labels.csv
Data loader initialized successfully.


In [32]:
# Evaluate the model and collect predictions

import torch.nn.functional as F
import numpy as np

all_labels, all_preds, all_probs = [], [], []

model.eval()
with torch.no_grad():
    for batch in loader:
        if batch is None:
            continue  # Skip invalid batch
        imgs, labels = batch
        outputs = model(imgs)
        probs = F.softmax(outputs, dim=1)[:, 1]
        preds = torch.argmax(outputs, dim=1)

        all_labels.extend(labels.numpy())
        all_preds.extend(preds.numpy())
        all_probs.extend(probs.numpy())

y_true = np.array(all_labels)
y_pred = np.array(all_preds)
y_prob = np.array(all_probs)
print(f"Collected {len(y_true)} valid samples for evaluation.")



Collected 2272 valid samples for evaluation.
